Esta cosa debería de tomar dos archivos, una sucesión de tiempos del GD y otra del CA, y checar coincidencias. Usemos Julia solamente y tratemos de paralelizar.

In [1]:
# using HDF5, JLD

In [1]:
ListaCA=readdir("./DatosEnero2016/espigas/CA3/");
ListaGD=readdir("./DatosEnero2016/espigas/DG/");

In [2]:
EntradasYValores=Dict{Any,Any}()
Randomizados=Dict{Any,Any}()

Dict{Any,Any} with 0 entries

In [3]:
ejemploCA=readdlm("./DatosEnero2016/CA3/Ch31_46.txt");
ejemploGD=readdlm("./DatosEnero2016/DG/Ch24_27.txt");

In [7]:
function CoincidenGDaCA(GD::Array, CA::Array)
    cuantosCA=length(CA)
    cuantosGD=length(GD)
    #Unidades en frames, son 1/7022 de segundo
    desde=13
    hasta=71
    coinciden::Int64=0
    

    j=1
    
    while j<cuantosGD
       
        k=1

        while k<cuantosCA
            dift=(CA[k]-GD[j])           
            if desde<dift<hasta
                coinciden+=1
            elseif dift>hasta
                break
            end
            k+=1        
        end
        j+=1       
    end
    
    return coinciden
end

CoincidenGDaCA (generic function with 1 method)

In [8]:
CoincidenGDaCA(ejemploGD,ejemploCA)

1

In [9]:
CoincidenciasCausales=Array(Int, 1,7)
Pupu=Int32[]
laregexp=r"([0-9]{2})_([0-9]{2})"
@time for k in ListaCA, j in ListaGD
    ejemploCA=readdlm("./DatosEnero2016/espigas/CA3/$k")
    ejemploGD=readdlm("./DatosEnero2016/espigas/DG/$j")  
    
    parapa=match(laregexp, k)
    perepe=match(laregexp,j)
    #println(typeof(parapa), parapa,k,j)
    
    parteCA=[parse(Int64,s) for s=parapa.captures ]  
    parteGD=[parse(Int64,z) for z=perepe.captures ]
    
    cuantos=CoincidenGDaCA(ejemploGD,ejemploCA)
    
    Pupu=[parteGD; parteCA; [cuantos; length(ejemploGD); length(ejemploCA)]]'
    CoincidenciasCausales=vcat(CoincidenciasCausales, Pupu)
    
end

CoincidenciasCausales=CoincidenciasCausales[2:end,:];


  5.217738 seconds (7.48 M allocations: 7.813 GB, 7.48% gc time)


In [10]:
writedlm("CoincidenciasPseudoRealesPureJulia01.dat", CoincidenciasCausales)

In [11]:
sum(CoincidenciasCausales[:,5])

9021

In [31]:
function sacaintervalos(xxs::Array)
    result=zeros(xxs)
    result[1]=xxs[1]
    for j=2:length(xxs)
        result[j]=xxs[j]-xxs[j-1]
    end
    return result
end

function tiempodeintervalos(xxs::Array)
    largo=length(xxs)
    result=zeros(largo)
    result[1]=xxs[1]
    for j=1:largo-1
        result[j+1]=result[j]+xxs[j+1]
    end
    return result
end
    

tiempodeintervalos (generic function with 1 method)

In [35]:
RandomCoincide=zeros(Float32, 1,7)
Pupu=Float32[]
cuantasveces=1000
promcoincide=0
@time for k in ListaCA, j in ListaGD
    
    ejemploCA=readdlm("./DatosEnero2016/CA3/$k")
    ejemploGD=readdlm("./DatosEnero2016/DG/$j")
    
    parapa=match(laregexp,k)
    perepe=match(laregexp,j)
    
    parteCA=[parse(Int64,s) for s=parapa.captures ]
    parteGD=[parse(Int64,s) for s=perepe.captures ]
        
    intervalosGD=sacaintervalos(vec(ejemploGD))
    intervalosCA=sacaintervalos(vec(ejemploCA))
    
    for t in 1:cuantasveces
        shuffle!(intervalosGD)
        shuffle!(intervalosCA)
        novoCA=tiempodeintervalos(intervalosCA)
        novoGD=tiempodeintervalos(intervalosGD)
        promcoincide+=CoincidenGDaCA(novoGD, novoCA)
        
    end
    promcoincide/=cuantasveces
    Pupu=[parteGD; parteCA; [promcoincide; length(ejemploGD); length(ejemploCA)]]'
    RandomCoincide=vcat(RandomCoincide, Pupu)

end

writedlm("RevueltasCoincidenPureJulia01.dat", RandomCoincide)

588.938330 seconds (85.69 M allocations: 45.378 GB, 0.69% gc time)


144-element Array{Float64,1}:
      1.49153e6
 273277.0      
      1.98415e6
      1.53654e6
 189339.0      
      1.58989e6
 935960.0      
 643957.0      
 658956.0      
      1.74382e6
 851744.0      
      1.66852e6
      2.07116e6
      ⋮        
  36267.0      
 112253.0      
 843196.0      
      1.31882e6
      1.51684e6
 717166.0      
 737108.0      
      1.60809e6
      1.48844e6
      1.98526e6
      1.47174e6
 723372.0      